In [1]:
import pandas as pd
from pathlib import Path
import numpy as np
import os
import matplotlib.pyplot as plt

In [2]:
Dataframe = pd.read_csv(os.path.join('sg138-19_C0038DLC_resnet50_StridulansUetziMar18shuffle1_200000_el_filtered.csv'))

In [3]:
Dataframe.head()

,coords,TR1.x,TR1.y,TR1.lh,MR1.x,MR1.y,MR1.lh,TiR1.x,TiR1.y,TiR1.lh,...,TiL1.lh,PL1.x,PL1.y,PL1.lh,FL1.x,FL1.y,FL1.lh,S.x,S.y,S.lh
0,0,1028.791,796.753,0.081,1019.750,771.745,0.406,NaN,NaN,0.0,...,0.420,NaN,NaN,0.0,NaN,NaN,0.000,621.702,138.918,0.982
1,1,1028.791,796.753,0.000,1019.750,772.167,0.158,NaN,NaN,0.0,...,0.865,NaN,NaN,0.0,NaN,NaN,0.000,625.570,139.088,0.995
2,2,1028.791,796.753,0.026,1020.604,772.167,0.157,NaN,NaN,0.0,...,0.202,NaN,NaN,0.0,NaN,NaN,0.000,625.843,139.997,0.994
3,3,1027.969,NaN,0.022,1020.513,772.167,0.083,NaN,NaN,0.0,...,0.085,NaN,NaN,0.0,1036.065,NaN,0.023,625.843,139.997,0.986
4,4,NaN,740.127,0.000,1019.750,772.271,0.146,NaN,NaN,0.0,...,0.708,NaN,NaN,0.0,NaN,12.115,0.000,625.843,139.979,0.994


In [4]:
Dataframe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46740 entries, 0 to 46739
Data columns (total 34 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   coords   46740 non-null  int64  
 1   TR1.x    43085 non-null  float64
 2   TR1.y    43103 non-null  float64
 3   TR1.lh   45646 non-null  float64
 4   MR1.x    42709 non-null  float64
 5   MR1.y    42691 non-null  float64
 6   MR1.lh   45646 non-null  float64
 7   TiR1.x   39809 non-null  float64
 8   TiR1.y   39796 non-null  float64
 9   TiR1.lh  45646 non-null  float64
 10  PR1.x    39200 non-null  float64
 11  PR1.y    39221 non-null  float64
 12  PR1.lh   45646 non-null  float64
 13  FR1.x    39725 non-null  float64
 14  FR1.y    39710 non-null  float64
 15  FR1.lh   45646 non-null  float64
 16  TL1.x    39953 non-null  float64
 17  TL1.y    39926 non-null  float64
 18  TL1.lh   45646 non-null  float64
 19  ML1.x    40523 non-null  float64
 20  ML1.y    40507 non-null  float64
 21  ML1.lh   456

In [5]:
df_melted_y = Dataframe.melt(id_vars=['coords'], 
                    value_vars=[col for col in Dataframe.columns if col.endswith('.y')],
                    var_name='bodypart', 
                    value_name='y')
df_melted_y.head()

,coords,bodypart,y
0,0,TR1.y,796.753
1,1,TR1.y,796.753
2,2,TR1.y,796.753
3,3,TR1.y,NaN
4,4,TR1.y,740.127


In [9]:
df_melted_lh = Dataframe.melt(id_vars=['coords'], 
                    value_vars=[col for col in Dataframe.columns if col.endswith('.lh')],
                    var_name='bodypart', 
                    value_name='lh')
df_melted_lh = df_melted_lh[df_melted_lh['lh'] > 0.9]
df_melted_lh.head()

,coords,bodypart,lh
93,93,TR1.lh,0.915
499,499,TR1.lh,0.919
500,500,TR1.lh,0.902
501,501,TR1.lh,0.947
502,502,TR1.lh,0.991


In [10]:
merged_df = pd.merge(df_melted_y, df_melted_lh, on=['coords'], how='left')
merged_df = merged_df.dropna()
merged_df = merged_df[merged_df['bodypart_x'].isin(['TR1.y', 'MR1.y', 'TL1.y', 'ML1.y', 'S.y'])]
merged_df.head()

,coords,bodypart_x,y,bodypart_y,lh
0,0,TR1.y,796.753,S.lh,0.982
1,1,TR1.y,796.753,S.lh,0.995
2,2,TR1.y,796.753,S.lh,0.994
4,4,TR1.y,740.127,S.lh,0.994
5,5,TR1.y,748.029,S.lh,0.998


In [11]:
plt.figure(figsize=(10, 6))

unique_bodyparts = merged_df['bodypart_x'].unique()
plt.rcParams.update({'font.size': 16})

# Assign colors (you can customize the colors here)
colors = plt.cm.Dark2(range(len(unique_bodyparts)))

# Loop through each bodypart and plot its corresponding line
for i, bodypart in enumerate(unique_bodyparts):
    # Filter the dataframe for the specific bodypart
    bodypart_data = merged_df[merged_df['bodypart_x'] == bodypart]
    
    # Plot the line for this bodypart
    plt.plot(bodypart_data['coords'], bodypart_data['y'], label=bodypart, color=colors[i])

# Add labels and title
plt.xlabel('Frame')
plt.ylabel('Y Coordinate')

# Show the legend
handles, labels = plt.gca().get_legend_handles_labels()
new_labels = ['Tarsus R', 'Metatarsus R', 'Tarsus L', 'Metatarsus L', 'Spinnerets'] 
plt.legend(handles, new_labels, loc="lower right")

plt.xlim(4000, 8000) 
plt.ylim(400, 900)

# Show the plot
plt.savefig('plot.png')
plt.close()